In [15]:
import os
interface_dir = os.environ["DATA"] + "webinterfaces/int04_prototype/"

tasks_dir = os.path.join(interface_dir, "res", "tasks")

results_dir = os.path.join(interface_dir, "results")
protocols_dir = os.path.join(interface_dir, "protocols")

results_filenames_d = {
    "CF" : ["TESTC3.json", "TESTC6.json", "TESTC10.json", "TESTC1.json"],
    "SHAP": ["TESTS3.json", "TESTS26.json"],
    "H": ["TESTH1.json"],
    "AI": ["TESTA2.json", "TESTA26.json"],
}

protocol_paths_d = {
    "CF": os.path.join(protocols_dir, "XAI_CF.json"),
    "SHAP": os.path.join(protocols_dir, "XAI_SHAP.json"),
    "H": os.path.join(protocols_dir, "H.json"),
    "AI": os.path.join(protocols_dir, "AI.json"),
}

EASY_TASKS = ["easy_3_7_purple", "easy_4_row_triangle", "easy_5_5_yellow", "easy_6_row_square"]
DIFFICULT_TASKS = ["hard_3_yellow_circle_plus_triangle_9", "hard_4_purple_squares_times2_circles", "hard_5_purple_triangle_plus_square_7", "hard_6_blue_circles_times2_triangles"]

TASK_KEYS = ["mainexp_easy_mild_1_task", "mainexp_easy_mild_2_task", "mainexp_easy_strong_1_task", "mainexp_easy_strong_2_task",
             "mainexp_hard_mild_1_task", "mainexp_hard_mild_2_task", "mainexp_hard_strong_1_task", "mainexp_hard_strong_2_task"]



In [16]:
import sys
sys.path.append("/home/jleguy/Documents/postdoc/git_repos/WebXAII/")

In [17]:
import json
import csv
import numpy as np

def load_json(path):
    with open(path) as json_file:
        return json.load(json_file)

def load_task_csv_file(path):
    y_true, y_pred = [], []
    with open(path) as csv_data:
        reader = csv.DictReader(csv_data)
        for row in reader:
            y_true.append(int(row["target"]))
            y_pred.append(int(row["pred"]))

    return np.array(y_true), np.array(y_pred)


In [21]:
from pywebxaii.resretrieval import extract_p_task_results

def compute_scores(results_dir, results_filenames_d, protocol_paths_d, tasks_dir, tasks_names, tasks_keys_results):

    output_res_d = {}

    # Iterating over all groups
    for group_key, filenames_list in results_filenames_d.items():

        output_res_d[group_key] = []

        # Iterating on all results files for the current group
        for filename in filenames_list:
            curr_res_path = os.path.join(results_dir, filename)

            nb_questions = 0
            nb_correct = 0
            early_break = False

            # Iterating over all tasks
            for task_idx, task_name in enumerate(tasks_names):

                task_true, _ =  load_task_csv_file(os.path.join(tasks_dir, task_name + "_content.csv"))

                answers_idx_vect, answers_text_vect, _, _, _ = \
                    extract_p_task_results(load_json(curr_res_path),
                                           tasks_keys_results[task_idx],
                                           protocol_d=load_json(protocol_paths_d[group_key]))

                nb_questions += len(answers_idx_vect)

                try:
                    nb_correct += np.sum(answers_idx_vect == np.logical_not(task_true))
                # Happens if the results file is not complete
                except ValueError:
                    output_res_d[group_key].append(None)
                    early_break = True
                    break

            if not early_break:
                output_res_d[group_key].append(nb_correct / nb_questions)

    return output_res_d


In [22]:
all_scores = compute_scores(results_dir, results_filenames_d, protocol_paths_d, tasks_dir, EASY_TASKS+DIFFICULT_TASKS,
                            TASK_KEYS)

In [23]:
all_scores

{'CF': [0.9821428571428571,
  0.9196428571428571,
  0.9553571428571429,
  0.8928571428571429],
 'SHAP': [0.9553571428571429, 0.9732142857142857],
 'H': [0.9642857142857143],
 'AI': [0.9375, 0.9642857142857143]}

In [33]:
def extract_quest_results(results_dir, results_filenames_d, protocol_paths_d, quest_keys):

    output_res_d = {}

    # Iterating over all groups
    for group_key, filenames_list in results_filenames_d.items():

        output_res_d[group_key] = {"raw": {}, "values": {}, "times": {}}

        # Iterating on all results files for the current group
        for filename in filenames_list:
            curr_res_path = os.path.join(results_dir, filename)

            # Iterating over all questionnaires keys
            for quest_key in quest_keys:

                answers_raw, answers_values, quest_times = extract_p_questionnaire_results(load_json(curr_res_path), quest_key,
                                                                                           protocol_d=load_json(protocol_paths_d[group_key]))
                if quest_key not in output_res_d[group_key]["raw"]:
                     output_res_d[group_key]["raw"][quest_key] = []
                     output_res_d[group_key]["values"][quest_key] = []
                     output_res_d[group_key]["times"][quest_key] = []

                output_res_d[group_key]["raw"][quest_key].append(answers_raw)
                output_res_d[group_key]["values"][quest_key].append(answers_values)
                output_res_d[group_key]["times"][quest_key].append(quest_times)

    return output_res_d

In [34]:
quest_keys = ["mainexp_easy_mild_cogload", "mainexp_easy_strong_cogload", "mainexp_hard_mild_cogload", "mainexp_hard_strong_cogload"]

cogload_results = extract_quest_results(results_dir, results_filenames_d, protocol_paths_d, quest_keys)

In [35]:
cogload_results

{'CF': {'raw': {'mainexp_easy_mild_cogload': [array([1, 0, 1, 1, 1, 0], dtype=object),
    array([3, 2, 2, 1, 1, 2], dtype=object),
    array([4, 1, 2, 1, 2, 1], dtype=object),
    array([4, 0, 1, 1, 5, 0], dtype=object)],
   'mainexp_easy_strong_cogload': [array([2, 0, 3, 2, 2, 1], dtype=object),
    array([3, 2, 4, 4, 2, 3], dtype=object),
    array([5, 2, 3, 1, 3, 1], dtype=object),
    array([5, 0, 4, 1, 4, 3], dtype=object)],
   'mainexp_hard_mild_cogload': [array([4, 1, 4, 4, 4, 2], dtype=object),
    array([4, 2, 3, 3, 4, 3], dtype=object),
    array([5, 3, 4, 3, 4, 4], dtype=object),
    array([6, 1, 4, 1, 5, 4], dtype=object)],
   'mainexp_hard_strong_cogload': [array([5, 2, 5, 3, 5, 2], dtype=object),
    array([4, 2, 4, 3, 4, 3], dtype=object),
    array([5, 3, 5, 4, 5, 5], dtype=object),
    array([5, 1, 5, 3, 5, 4], dtype=object)]},
  'values': {'mainexp_easy_mild_cogload': [array(['', 'Very low', '', '', '', 'Very low'], dtype=object),
    array(['', '', '', '', '', ''], 

In [ ]:
quest_keys = ["mainexp_easy_mild_cogload", "mainexp_easy_strong_cogload", "mainexp_hard_mild_cogload", "mainexp_hard_strong_cogload"]

cogload_results = extract_quest_results(results_dir, results_filenames_d, protocol_paths_d, quest_keys)

In [12]:
cogload_easy_mild_cf = [extract_p_questionnaire_results(load_json_file(os.path.join(results_dir, results_cf[i])),
                                                        "mainexp_easy_mild_cogload",
                                                        load_json_file(protocol_cf_path)) for i in range(len(results_cf))]

cogload_hard_strong_cf = [extract_p_questionnaire_results(load_json_file(os.path.join(results_dir, results_cf[i])),
                                                        "mainexp_hard_strong_cogload",
                                                        load_json_file(protocol_cf_path)) for i in range(len(results_cf))]

In [13]:
cogload_easy_mild_cf

[(array([1, 0, 1, 1, 1, 0], dtype=object),
  array(['', 'Very low', '', '', '', 'Very low'], dtype=object),
  72015),
 (array([4, 1, 2, 1, 2, 1], dtype=object),
  array(['', '', '', '', '', ''], dtype=object),
  34301),
 (array([4, 0, 1, 1, 5, 0], dtype=object),
  array(['', 'Very low', '', '', '', 'Very low'], dtype=object),
  36033)]

In [14]:
cogload_hard_strong_cf

[(array([5, 2, 5, 3, 5, 2], dtype=object),
  array(['', '', '', '', '', ''], dtype=object),
  20797),
 (array([5, 3, 5, 4, 5, 5], dtype=object),
  array(['', '', '', '', '', ''], dtype=object),
  20465),
 (array([5, 1, 5, 3, 5, 4], dtype=object),
  array(['', '', '', '', '', ''], dtype=object),
  23495)]